In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
face_vector = []
for i in range(151):
    face_image = cv2.cvtColor(cv2.imread(f"./dataset/{str(i).zfill(5)}.png"), cv2.COLOR_RGB2GRAY)
    face_image = face_image.reshape(128*128,)
    face_vector.append(face_image)
face_vector = np.asarray(face_vector)
face_vector = face_vector.transpose()

print(face_vector.shape)
print(face_vector)

(16384, 151)
[[120   1 181 ... 141 122 154]
 [118   1 182 ... 150 123 143]
 [116   1 182 ... 150 129 155]
 ...
 [159 120  70 ... 164 112  89]
 [155 119  69 ... 166 112 139]
 [156 122  64 ... 156 115 209]]


In [3]:
#STEP2: Normalize the face vectors by calculating the average face vector and subtracting it from each vector
avg_face_vector = face_vector.mean(axis=1)
avg_face_vector = avg_face_vector.reshape(face_vector.shape[0], 1)
normalized_face_vector = face_vector - avg_face_vector
print(normalized_face_vector)

[[   3.01986755 -115.98013245   64.01986755 ...   24.01986755
     5.01986755   37.01986755]
 [   1.37748344 -115.62251656   65.37748344 ...   33.37748344
     6.37748344   26.37748344]
 [  -1.25165563 -116.25165563   64.74834437 ...   32.74834437
    11.74834437   37.74834437]
 ...
 [  53.15231788   14.15231788  -35.84768212 ...   58.15231788
     6.15231788  -16.84768212]
 [  48.00662252   12.00662252  -37.99337748 ...   59.00662252
     5.00662252   32.00662252]
 [  48.02649007   14.02649007  -43.97350993 ...   48.02649007
     7.02649007  101.02649007]]


In [4]:
#STEP3: Calculate the Covariance Matrix or the Sigma
covariance_matrix = np.cov(np.transpose(normalized_face_vector))
# covariance_matrix = np.transpose(normalized_face_vector).dot(normalized_face_vector)
print(covariance_matrix)

[[3050.6339385  -142.9067319    28.53081304 ...  555.13357034
  -168.90027751   92.73587738]
 [-142.9067319  1933.35589488 -314.36290238 ...  -55.17640393
   246.37821502  152.25877194]
 [  28.53081304 -314.36290238 1955.76067469 ... -501.24000451
   411.70287537 -568.63986091]
 ...
 [ 555.13357034  -55.17640393 -501.24000451 ... 3439.75954751
  -666.38022593 -843.13573075]
 [-168.90027751  246.37821502  411.70287537 ... -666.38022593
  2314.18974549  217.45649339]
 [  92.73587738  152.25877194 -568.63986091 ... -843.13573075
   217.45649339 2579.07860606]]


In [5]:
#STEP4: Calculate Eigen Vectors
eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)

In [6]:
#STEP5: Select the K best Eigen Faces, K < M
print(eigen_vectors.shape)
k = 20
k_eigen_vectors = eigen_vectors[0:k, :]
print(k_eigen_vectors.shape)

(151, 151)
(20, 151)


In [7]:
#STEP6: Convert lower dimensionality K Eigen Vectors to Original Dimensionality
eigen_faces = k_eigen_vectors.dot(np.transpose(normalized_face_vector))
print(eigen_faces.shape)

(20, 16384)


In [8]:
# STEP7: Represent Each eigen face as combination of the K Eigen Vectors
# weights = eigen_faces.dot(normalized_face_vector)
weights = np.transpose(normalized_face_vector).dot(np.transpose(eigen_faces))
print(weights)

[[  7710014.08164914 -13958115.74293288  19572129.28368166 ...
    2680929.78302859  20805322.15677225   9502800.1843103 ]
 [-38532568.05683067  54989474.07400095 -31091984.72452674 ...
  -33243228.90600804  -5877962.44735014 -19253897.7076951 ]
 [  7963955.23258606   3789591.1250895     265913.69135887 ...
   12494441.9301071     821790.9627331   14178500.60301809]
 ...
 [-28008285.73885257  33532463.99402824   7163945.85501368 ...
   -4892990.95496545  17998925.98127977  -8863376.510942  ]
 [  9067953.98854884   8035211.9882185  -19014900.51359725 ...
   -7213020.734944      322338.180603    -2568405.22892493]
 [ 39109290.41759067 -60023490.00472989  14208406.4660981  ...
    6287224.21582842   -992278.81332618   -693052.78898194]]


In [11]:
#STEP8: Testing Phase
test_add = f"./dataset/00149.png"
test_img = cv2.imread(test_add)
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2GRAY)


test_img = test_img.reshape(128*128, 1)
test_normalized_face_vector = test_img - avg_face_vector
test_weight = np.transpose(test_normalized_face_vector).dot(np.transpose(eigen_faces))


index =  np.argmin(np.linalg.norm(test_weight - weights, axis=1))

print(index);

149
